### Import libraries

In [36]:
from dotenv import load_dotenv, find_dotenv
import os
from langchain_community.vectorstores import FAISS
from langchain_ai21 import AI21Embeddings, ChatAI21
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [37]:
load_dotenv(find_dotenv())

True

### Load documents

In [38]:
job_description_data = [
    """Java developer roles will vary greatly depending on companies and job positions. Here are some typical roles and responsibilities of java developers:
Contribute to all stages of software development lifecycle
Design, implement and maintain Java-based applications that can be high-volume and low-latency
Analyze user requirements to define business objectives
Envisioning system features and functionality
Define application objectives and functionality
Ensure application designs conform with business goals
Develop and test software
Identify and resolve any technical issues arising
Create detailed design documentation
Propose changes to current Java infrastructure
Develop technical designs for application development
Develop multimedia applications
Write well designed, testable code
Conducting software analysis, programming, testing, and debugging
Manage Java and Java EE application development
Develop documentation to help users
Transforming requirements into stipulations
Prepare and produce releases of software components
Support continuous improvement, investigating alternatives and technologies, and presenting for architectural review""",
    """Job brief
We are looking for a Python Developer to join our engineering team and help us develop and maintain various software products.

Python Developer responsibilities include writing and testing code, debugging programs and integrating applications with third-party web services. To be successful in this role, you should have experience using server-side logic and work well in a team.

Ultimately, you’ll build highly responsive web applications that align with our business needs.

Responsibilities
Write effective, scalable code
Develop back-end components to improve responsiveness and overall performance
Integrate user-facing elements into applications
Test and debug programs
Improve functionality of existing systems
Implement security and data protection solutions
Assess and prioritize feature requests
Coordinate with internal teams to understand user requirements and provide technical solutions
Requirements and skills
Work experience as a Python Developer
Expertise in at least one popular Python framework (like Django, Flask or Pyramid)
Knowledge of object-relational mapping (ORM)
Familiarity with front-end technologies (like JavaScript and HTML5)
Team spirit
Good problem-solving skills
BSc in Computer Science, Engineering or relevant field
    """,
    """
React.js Developer Responsibilities:
Meeting with the development team to discuss user interface ideas and applications.
Reviewing application requirements and interface designs.
Identifying web-based user interactions.
Developing and implementing highly responsive user interface components using react concepts.
Writing application interface codes using JavaScript following react.js workflows.
Troubleshooting interface software and debugging application codes.
Developing and implementing front-end architecture to support user interface concepts.
Monitoring and improving front-end performance.
Documenting application changes and developing updates.
React.js Developer Requirements:
Bachelor’s degree in computer science, information technology, or a similar field.
Previous experience working as a react.js developer.
In-depth knowledge of JavaScript, CSS, HTML, and front-end languages.
Knowledge of REACT tools including React.js, Webpack, Enzyme, Redux, and Flux.
Experience with user interface design.
Knowledge of performance testing frameworks including Mocha and Jest.
Experience with browser-based debugging and performance testing software.
Excellent troubleshooting skills.
Good project management skills.
    """
]

In [39]:
job_categories = ["Java", "Python", "React"]

In [40]:
# Create Document from job description data
job_description_docs = [Document(page_content=job_description, metadata={"category": job_category}) 
                        for job_description, job_category in zip (job_description_data, job_categories)
                        ]

job_description_docs

[Document(page_content='Java developer roles will vary greatly depending on companies and job positions. Here are some typical roles and responsibilities of java developers:\nContribute to all stages of software development lifecycle\nDesign, implement and maintain Java-based applications that can be high-volume and low-latency\nAnalyze user requirements to define business objectives\nEnvisioning system features and functionality\nDefine application objectives and functionality\nEnsure application designs conform with business goals\nDevelop and test software\nIdentify and resolve any technical issues arising\nCreate detailed design documentation\nPropose changes to current Java infrastructure\nDevelop technical designs for application development\nDevelop multimedia applications\nWrite well designed, testable code\nConducting software analysis, programming, testing, and debugging\nManage Java and Java EE application development\nDevelop documentation to help users\nTransforming requir

In [41]:
# split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(job_description_docs)
docs

[Document(page_content='Java developer roles will vary greatly depending on companies and job positions. Here are some typical roles and responsibilities of java developers:\nContribute to all stages of software development lifecycle\nDesign, implement and maintain Java-based applications that can be high-volume and low-latency\nAnalyze user requirements to define business objectives\nEnvisioning system features and functionality\nDefine application objectives and functionality\nEnsure application designs conform with business goals\nDevelop and test software\nIdentify and resolve any technical issues arising\nCreate detailed design documentation\nPropose changes to current Java infrastructure\nDevelop technical designs for application development\nDevelop multimedia applications\nWrite well designed, testable code\nConducting software analysis, programming, testing, and debugging\nManage Java and Java EE application development\nDevelop documentation to help users\nTransforming requir

### Create embedding model

In [42]:
embeddings = AI21Embeddings()

### Create vector store

In [43]:
VECTOR_STORE_PATH = "data/job_description_db"
try:
    # try to load the vector store if it exists
    vector_store = FAISS.load_local(VECTOR_STORE_PATH)
except:
    # if it doesn't exist, create a new vector store
    vector_store = FAISS.from_documents(docs, embeddings)
    # save the vector store to disk
    vector_store.save_local(VECTOR_STORE_PATH)

# total of vectors in the vector store
vector_store.index.ntotal

6

### Retrieving relevant documents

In [47]:
query = "5 years of experience as role of Python backend developer. My tech stack includes FastAPI, Flask. I am looking for a new job."
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
retrived_docs = retriever.invoke(query)
retrived_docs

[Document(page_content='Ultimately, you’ll build highly responsive web applications that align with our business needs.\n\nResponsibilities\nWrite effective, scalable code\nDevelop back-end components to improve responsiveness and overall performance\nIntegrate user-facing elements into applications\nTest and debug programs\nImprove functionality of existing systems\nImplement security and data protection solutions\nAssess and prioritize feature requests\nCoordinate with internal teams to understand user requirements and provide technical solutions\nRequirements and skills\nWork experience as a Python Developer\nExpertise in at least one popular Python framework (like Django, Flask or Pyramid)\nKnowledge of object-relational mapping (ORM)\nFamiliarity with front-end technologies (like JavaScript and HTML5)\nTeam spirit\nGood problem-solving skills\nBSc in Computer Science, Engineering or relevant field', metadata={'category': 'Python'}),
 Document(page_content='In-depth knowledge of Ja

In [49]:
retrived_docs[0].metadata    

{'category': 'Python'}

In [52]:
vector_store.similarity_search_with_score(query, k=3)

[(Document(page_content='Ultimately, you’ll build highly responsive web applications that align with our business needs.\n\nResponsibilities\nWrite effective, scalable code\nDevelop back-end components to improve responsiveness and overall performance\nIntegrate user-facing elements into applications\nTest and debug programs\nImprove functionality of existing systems\nImplement security and data protection solutions\nAssess and prioritize feature requests\nCoordinate with internal teams to understand user requirements and provide technical solutions\nRequirements and skills\nWork experience as a Python Developer\nExpertise in at least one popular Python framework (like Django, Flask or Pyramid)\nKnowledge of object-relational mapping (ORM)\nFamiliarity with front-end technologies (like JavaScript and HTML5)\nTeam spirit\nGood problem-solving skills\nBSc in Computer Science, Engineering or relevant field', metadata={'category': 'Python'}),
  0.30979526),
 (Document(page_content='In-dept